In [3]:
import re
import matplotlib.pyplot as plt
%matplotlib inline  
import numpy as np
import pandas as pd
import json
import csv
import sys
import math


numLines = 0
vcount, fcount = 0, 0
vlist, flist, olist = [], [], []
flag = 0
with open("../data/mouse03.obj", "r") as fin:
    for line in fin:
        numLines = numLines + 1
        tokens = line.split()
        if (tokens[0] == 'o'):
            name = tokens[1]
            olist.append(name.lower())
            if (flag == 1):
                vlist.append(vcount)
                flist.append(fcount)
                flag = 0
            flag = 1
            vcount = 0
            fcount = 0
        elif (tokens[0] == 'v'):
            vcount = vcount + 1
        elif (tokens[0] == 'f'):
            fcount = fcount + 1    
vlist.append(vcount)
flist.append(fcount)
print len(olist)
print len(vlist)
print len(flist)

df_obj = pd.DataFrame({'name': olist, 'vcount': vlist, 'fcount': flist})
df_obj

6868
6868
6868


,fcount,name,vcount
0,1676324,astro1,836292
1,960,ellipsoid289_sphere.1341,482
2,264,glycogen.3536_anything/[ball_--1355]_--1355.3536,134
3,264,glycogen.3452_anything/[ball_--1355]_--1355.3452,134
4,264,glycogen.3451_anything/[ball_--1355]_--1355.3451,134
5,960,ellipsoid288_sphere.1339,482
6,264,glycogen.1800_anything/[ball_--1355]_--1355.1800,134
7,264,glycogen.1786_anything/[ball_--1355]_--1355.1786,134
8,264,glycogen.1780_anything/[ball_--1355]_--1355.1780,134
9,960,ellipsoid287_sphere.1337,482


In [5]:
# read hvgx file
sg = {}
SG = []
parent = ''
with open("../data/mouse3_metadata.hvgx", "r") as fin:
    for line in fin:
        tokens = line.split(',')
        if (tokens[0] == 'sg'): 
            #12) sg: segmentation metadata
            # vast_id, flags, r,g,b, pattern1, r2, g2, b2, pattern2, anchorx, anchory, anchorz, 
            # parent_id, child_id, previous_sibiling_id, next_sibiling_id, collapsed, bboxx1, 
            # bboxy1, bboxz1, bboxx2, bboxy2, bboxz2, voxels, type, object_id, name
            name = tokens[-1][:-2]
            if name[:3] == 'bou' or name[:3] == 'spi':
                name_tok = name.split('.')
                name = name + "." + str(parent)
            elif name[:3] == 'den':
                parent =  name
            elif name[:3] == 'axo':
                parent = name
                
            if (bool(sg.get(name))):
                print "dup: ", name
                
            # remove \r\n from name
            tokens[-1] = tokens[-1][:-2]  
                
            sg[name] = tokens[1:]
            
            SG.append(tokens[1:])
        
print "sg (segmentation metadata): " + str(len(sg))

sg (segmentation metadata): 1100


In [10]:
df_sg = pd.DataFrame(SG, columns = ['vast_id', 'flags', 'r', 'g', 'b', 'pattern1', 'r2', 'g2', 'b2', 
                                    'pattern2', 'anchorx', 'anchory', 'anchorz', 
                                    'parent_id', 'child_id', 'previous_sibiling_id', 
                                    'next_sibiling_id', 'collapsed', 'bboxx1', 
                                    'bboxy1', 'bboxz1', 'bboxx2', 'bboxy2', 'bboxz2', 
                                    'voxels', 'type', 'object_id', 'name'])
df_sg['name'] = df_sg['name'].apply(lambda x: x[:-2])
df_sg.drop(df_sg.columns[[5, 6, 7, 8, 9, 10, 11, 12, 18, 19, 20, 21, 22, 23, 24]], axis=1, inplace=True)
print df_sg.type.unique()
df_sg

['' 'DENDRITE' 'SPINE' 'AXON' 'BOUTON' 'GLIA' 'SYNAPSE' 'MITOCHONDRIA']


,vast_id,flags,r,g,b,parent_id,child_id,previous_sibiling_id,next_sibiling_id,collapsed,type,object_id,name
0,10000,0,0,0,0,0,10001,0,0,0,,,Struc
1,10002,0,0,0,0,10000,1,10001,10003,0,,,Dendrit
2,1,0,252,187,161,10002,356,0,2,0,DENDRITE,1,dendrite056_dendrite1
3,356,0,253,208,162,1,0,0,0,0,SPINE,356,spine1_in_sol
4,2,0,252,146,114,10002,357,1,3,0,DENDRITE,2,dendrite082_dendrite1
5,357,0,253,174,60,2,0,0,0,0,SPINE,357,spine1_sol
6,3,0,252,187,161,10002,358,2,7,0,DENDRITE,3,dendrite081_dendrite1
7,358,0,253,174,60,3,0,0,0,0,SPINE,358,spine1_sol
8,10001,0,0,0,0,10000,4,0,10002,0,,,Axo
9,4,0,8,81,156,10001,359,0,5,0,AXON,4,axon091_axon3


In [11]:
header = ['hvgxId', 'hvgxName', 'objName']
hvgxToObj = {}

parent = -1
parentName = ''
sbCounter = 0
for obj in olist:
    # clean the name
    # check in sg metada
    key = obj
    code =  obj[:3]
    tokens = re.findall(r"([\w_']*)", obj)
    tokens = tokens[0].split('_')
    if ("surf" in tokens or "curve" in tokens or "surface" in tokens or 'length' in tokens or 'sphere' in tokens or 'submesh' in tokens): 
        continue

    tokens = obj.split('.')    
    key = tokens[0]

    if (code[0] == 'd' and code[1] != 'e'):
        # synapse
        key = obj.split('[')[0]
    elif (code == 'bou'):
        k = 0
        key = tokens[0] +"."+ pAxonName
    elif (code == 'spi'):
        key = tokens[0] +"."+ pDedriteName
    elif (code == 'gly'):
        continue
        
    if (not bool(sg.get(key))):
        print 'DIDNT FIND IN SG', key
        continue
      
    if (code == 'spi'):
        hvgxID = sg[key][26]
        parent = sg[key][13]
        name = sg[key][27][:-2]
             
    elif (code == 'bou'):
        hvgxID = sg[key][26]
        parent = sg[key][13]
        name = sg[key][27][:-2]

    elif (code == 'den'):
        pDendriteID = sg[key][26]
        pDedriteName = sg[key][27]
    elif (code == 'axo'):
        pAxonID = sg[key][26]
        pAxonName = sg[key][27]
    
    hvgxToObj[obj] = sg[key]

DIDNT FIND IN SG axon092_e_lenght
DIDNT FIND IN SG dendrite049_maybe_axon060_dendrites70
DIDNT FIND IN SG m003_solid
DIDNT FIND IN SG m008_solid
DIDNT FIND IN SG m010_solid
DIDNT FIND IN SG m011_solid
DIDNT FIND IN SG m012_solid
DIDNT FIND IN SG m013_solid
DIDNT FIND IN SG m014_solid
DIDNT FIND IN SG m018_solid
DIDNT FIND IN SG m019_solid
DIDNT FIND IN SG m021_solid
DIDNT FIND IN SG m022_solid
DIDNT FIND IN SG m023_solid
DIDNT FIND IN SG m024_solid
DIDNT FIND IN SG m025_solid
DIDNT FIND IN SG m026_solid
DIDNT FIND IN SG m027_solid
DIDNT FIND IN SG m028_solid
DIDNT FIND IN SG m030_solid
DIDNT FIND IN SG m031_solid
DIDNT FIND IN SG m033_solid
DIDNT FIND IN SG m035_solid
DIDNT FIND IN SG m036_solid
DIDNT FIND IN SG m037_solid
DIDNT FIND IN SG m038_solid
DIDNT FIND IN SG m041_solid
DIDNT FIND IN SG m042_solid
DIDNT FIND IN SG m044_solid
DIDNT FIND IN SG m045_solid
DIDNT FIND IN SG m046_solid
DIDNT FIND IN SG m047_solid
DIDNT FIND IN SG m048_solid
DIDNT FIND IN SG m049_solid
DIDNT FIND IN S

In [9]:
df_hvgxToObj = pd.DataFrame(hvgxToObj)
df_hvgxToObj

,astro01_astro1,astro1,axon001_e_axon1_e_def_19_04_1352,axon002_e_axons02,axon003_i_axon02.001_i_m_3_synapses_in_3d_part1_14,axon004_i_axons03_i_axons03,axon005_e_axons04.001_e_axons04.001,axon006_e_axons04_e_axons04,axon007_i_axons05_i_axons05,axon008_axons06,...,syn_d080s01a266b01_e_psd136,syn_d081s01a091b01_psd109,syn_d082s01a048b01_psd142,syn_dxxxsxxa002b01_e_psd161,syn_dxxxsxxa093b01_psd165,syn_dxxxsxxa139b01_e_psd009,syn_dxxxsxxa229b01_e_psd126,syn_dxxxsxxa273b01_u_psd085,syn_dxxxsxxa274b01_psd133,syn_xxxnotsynapse_d69s00a55b2_e_psd055
0,746,745,158,159,160,161,162,163,164,165,...,882,855,888,907,911,755,872,831,879,801
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35,116,66,158,8,8,33,33,198,66,...,247,247,247,247,247,247,247,247,247,247
3,139,196,146,202,81,81,113,113,219,146,...,104,104,104,104,104,104,104,104,104,104
4,69,118,198,225,156,156,181,181,239,198,...,161,161,161,161,161,161,161,161,161,161
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
obj_properties = []
paxon = -1
pden = -1
for index, row in df_glyObj.iterrows():
    key = row['oname'].lower()
    if ( not bool(hvgxToObj.get(key))):
        continue
    Id = hvgxToObj[key][0]
    name = hvgxToObj[key][-1]
    type = hvgxToObj[key][-3]
    # 'oname', 'distance', 'glyId', 'clusters', 'count'
    glyId = row['glyId']
    clusters = row['clusters']
    count = row['count']
     
    exitory_inhibit = -1
    if (bool(axons.get(name))):
        exitory_inhibit = axons[name][1]
        paxon = exitory_inhibit
    elif (bool(dn.get(name))):
        exitory_inhibit = dn[name][1]
        pden = exitory_inhibit
    #   get the volume from these: Allpixels, Astropixels, ADpixels, NSpixels, Spixels
    # and bool(Spixels.get(str(Id)))
    if type == 'SYNAPSE' and bool(Spixels.get(str(Id))):
        vol =  Spixels.get(str(Id))
    elif (type == 'SPINE' or type == 'BOUTON') and bool(NSpixels.get(str(Id))):
        vol =  NSpixels.get(str(Id))
        if (type == 'SPINE'):
            exitory_inhibit  =  pden
        else:
            exitory_inhibit  =  paxon
    elif (type == 'AXON' or type == 'DENDRITE') and bool(ADpixels.get(str(Id))):
        vol =  ADpixels.get(str(Id))
    elif ( bool(Allpixels.get(str(Id)))):
        vol =  Allpixels.get(str(Id))
    #print Id, name, type, distance, glyId, clusters, count
    obj_properties.append([int(Id), name, key, type, glyId, clusters, count, vol, int(exitory_inhibit)])

In [ ]:
df_obj_properties = pd.DataFrame(obj_properties, columns=['Id', 'hvgxName', 'objName', 'type', 
                                                          'glyId', 'clusters', 'count', 'volume', 'exitory_inhibit'])
df_obj_properties.to_csv('m3_df_obj_properties.csv')